In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

train_X = np.linspace(-1,1,100)#x轴数据点
test_X = np.linspace(-0.5,0.8,30)#生成测试数据

train_Y = 100 * train_X + np.random.randn(train_X.shape[0])*0.3#这个数据必须是由一定误差的，不然就过拟合了，没有意义了

plt.plot(train_X,train_Y,'ro',label='Original data')
plt.legend()
plt.show()

X = tf.placeholder('float')
Y = tf.placeholder('float')
#模型参数
W = tf.Variable(tf.random_normal([1]),name='Weight')#随机生成-1 1之间的正太分布的值
b = tf.Variable(tf.zeros([1]),name='Bias')
z = tf.multiply(X,W) + b
cost = tf.reduce_mean(tf.square(Y-z))
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
def moving_average(a,w=10):
    if len(a) < w:
        return a[:]
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx,val in enumerate(a)]
init = tf.global_variables_initializer()
#定义参数
training_epochs = train_X.size
display_step = 10
plotdata = {'batchsize':[],'loss':[]}
#启动session会话
with tf.Session() as sess:
    sess.run(init)
    #想模型输入数据
    for epoch in range(training_epochs):
        for (x,y) in zip(train_X,train_Y):
            sess.run(optimizer,feed_dict={X:x,Y:y})
        #显示训练的详细信息
        if epoch % display_step == 0:
            loss = sess.run(cost,feed_dict={X:train_X,Y:train_Y})
            print('Epoch:',epoch + 1,'loss=',loss,'W=',sess.run(W),'b=',sess.run(b))
            if not (loss == 'NA'):
                plotdata['batchsize'].append(epoch)
                plotdata['loss'].append(loss)
    print('Model training Fnished')
    print('cost = ',sess.run(cost,feed_dict={X:train_X,Y:train_Y}),'W=',sess.run(W),'b=',sess.run(b))
    print('#'*20,'使用模型','#'*20)
    for x in zip(test_X):
        print('x =',x,'z=',sess.run(z,feed_dict={X:x}))

#x训练图可视化
    plt.plot(train_X,train_Y,'ro',label='Original data')
    plt.plot(train_X,sess.run(W)*train_X + sess.run(b),label='Fittedline')
    #测试数据显示
    plt.plot(test_X,sess.run(W)*test_X + sess.run(b),'co',label='Testline')
    plt.legend()
    plt.show()
    
    plotdata['avgloss'] = moving_average(plotdata['loss'])
    print('plotdata[avgloss]:',plotdata['avgloss'])
    plt.figure(1)
    plt.subplot(211)
    plt.plot(plotdata['batchsize'],plotdata['avgloss'],'b--')
    plt.xlabel('Minibatch number')
    plt.ylabel('Loss')
    plt.title('Minibatch run vs Training loss')
    plt.show()